##  *Statistics coded:* Causes of death statistics

Prepared by  [**Katharina Koehler**](@kkatha) (EMOS).

Objective of this notebook is illustrating the "Statistics Explained" article [Causes of death statisitics](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Causes_of_death_statistics).

So, indeed, you can choose to create a notebook in your own language. Take into account that not all articles will be available in the language you choose, but you can still decide to "translate" the English source. This is left to you.

Note: please indicate the name of the programming language in the name of the notebook, as well as the language of the text used in the notebook, *e.g.* using the extensions *_fr_py.ipynb* or *_de_r.ipynb*.

### Configuration

Put all libraries/packages/modules/etc... that are necessary so that you can run this notebook.

In [0]:
# install.packages(...)
# library(...)

### Overview/ Introduction

Creation of graphic: causes of death - standardised death rate, EU-28, 2016

In [ ]:
# your code

### Main findings

The code here will help reproducing the material presented in the section [Main findings](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Causes_of_death_statistics).

In [ ]:
# your code

Don't forget to explain what you are doing...

### Additional material

At this stage, it is up to you to extend the work presented in the notebook with further analysis/visualisation you deem relevant. 